In [64]:
import torch
import importlib
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
importlib.reload(__import__('helpers'))
from helpers import save_fewshot_results

In [28]:
CONFIG = {
    'n_way': 3,           # Número de classes por episódio
    'n_shot': 5,          # Exemplos de treino por classe
    'n_query': 15,        # Exemplos de teste por classe
    'n_episodes': 200,    # Número de episódios para avaliação
    'model_name': 'deit_base_distilled_patch16_224',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

In [29]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   
        std=[0.229, 0.224, 0.225]
    )
])

In [30]:
from torchvision import datasets
from collections import defaultdict

dataset = datasets.ImageFolder(
    root='/content/drive/MyDrive/pdi/dataset/ham10000/all',
    transform=transform
)

# mapa classe → índices
class_to_indices = defaultdict(list)
for idx, (_, label) in enumerate(dataset):
    class_to_indices[label].append(idx)


In [31]:
import random
import torch
from torch.utils.data import Subset, DataLoader

def create_episode(
    class_to_indices,
    n_way=5,
    n_shot=10,
    n_query=15
):
    classes = random.sample(list(class_to_indices.keys()), n_way)

    support_idx = []
    query_idx = []

    for c in classes:
        indices = random.sample(
            class_to_indices[c],
            n_shot + n_query
        )
        support_idx += indices[:n_shot]
        query_idx   += indices[n_shot:]

    return support_idx, query_idx, classes


In [32]:
def get_episode_loaders(dataset, support_idx, query_idx):
    support_loader = DataLoader(
        Subset(dataset, support_idx),
        batch_size=len(support_idx),
        shuffle=False
    )

    query_loader = DataLoader(
        Subset(dataset, query_idx),
        batch_size=len(query_idx),
        shuffle=False
    )

    return support_loader, query_loader


In [33]:
def extract_features_vit(loader, model):
    features = []
    labels = []

    model.eval()
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(next(model.parameters()).device)
            feats = model(imgs)      # saída do DeiT sem head
            features.append(feats)
            labels.append(lbls)

    return torch.cat(features), torch.cat(labels)


In [34]:
import torch.nn.functional as F

def evaluate_episode(
    model,
    dataset,
    class_to_indices,
    device,
    n_way=5,
    n_shot=10,
    n_query=15
):
    support_idx, query_idx, classes = create_episode(
        class_to_indices, n_way, n_shot, n_query
    )

    support_loader, query_loader = get_episode_loaders(
        dataset, support_idx, query_idx
    )

    support_features, support_labels = extract_features_vit(
        support_loader, model
    )
    query_features, query_labels = extract_features_vit(
        query_loader, model
    )

    # Guardar labels originais antes do remapeamento
    original_query_labels = query_labels.cpu().numpy()

    # normalização
    support_features = F.normalize(support_features, p=2, dim=1)
    query_features   = F.normalize(query_features, p=2, dim=1)

    # remapeia rótulos para [0..n_way-1]
    label_map = {c: i for i, c in enumerate(classes)}
    support_labels = torch.tensor(
        [label_map[int(l)] for l in support_labels],
        device=device
    )
    query_labels_remapped = torch.tensor(
        [label_map[int(l)] for l in query_labels],
        device=device
    )

    # protótipos
    prototypes = torch.zeros(n_way, support_features.size(1)).to(device)
    for i in range(n_way):
        prototypes[i] = support_features[support_labels == i].mean(0)
    
    # normalizar todos os protótipos
    prototypes = F.normalize(prototypes, p=2, dim=1)

    # similaridade
    sims = torch.mm(query_features, prototypes.t())
    preds_remapped = sims.argmax(dim=1)

    # Acurácia do episódio (com labels remapeados)
    acc = (preds_remapped == query_labels_remapped).float().mean().item()
    
    # Reverter predições para classes originais
    original_preds = np.array([classes[int(p)] for p in preds_remapped.cpu().numpy()])
    
    return acc, preds_remapped.cpu().numpy(), query_labels_remapped.cpu().numpy(), original_preds, original_query_labels


In [ ]:
import timm
import torch

print(CONFIG['device'])
model = timm.create_model(
    CONFIG['model_name'],
    pretrained=True,
    num_classes=0
)

model = model.to(CONFIG['device'])
model.eval()


In [36]:
import numpy as np

accuracies = []
# Labels remapeados (para cada episódio independente)
all_predictions_remapped = []
all_labels_remapped = []
# Labels originais (classes reais do dataset)
all_predictions_original = []
all_labels_original = []

model.eval()

for ep in range(CONFIG['n_episodes']):
    acc, preds_remapped, labels_remapped, preds_original, labels_original = evaluate_episode(
        model,
        dataset,
        class_to_indices,
        CONFIG['device'],
        n_way=CONFIG['n_way'],
        n_shot=CONFIG['n_shot'],
        n_query=CONFIG['n_query']
    )
    accuracies.append(acc)
    # Armazenar ambas as versões
    all_predictions_remapped.extend(preds_remapped)
    all_labels_remapped.extend(labels_remapped)
    all_predictions_original.extend(preds_original)
    all_labels_original.extend(labels_original)
    print(f"Episódio {ep+1}: {acc*100:.2f}%")

mean_acc = np.mean(accuracies)
std_acc  = np.std(accuracies)

print(f"\nAcurácia final: {mean_acc*100:.2f}% ± {std_acc*100:.2f}%")

Episódio 1: 82.22%
Episódio 2: 53.33%
Episódio 3: 66.67%
Episódio 4: 44.44%
Episódio 5: 62.22%
Episódio 6: 97.78%
Episódio 7: 60.00%
Episódio 8: 73.33%
Episódio 9: 68.89%
Episódio 10: 64.44%
Episódio 11: 71.11%
Episódio 12: 64.44%
Episódio 13: 84.44%
Episódio 14: 44.44%
Episódio 15: 77.78%
Episódio 16: 68.89%
Episódio 17: 68.89%
Episódio 18: 73.33%
Episódio 19: 71.11%
Episódio 20: 48.89%
Episódio 21: 73.33%
Episódio 22: 75.56%
Episódio 23: 64.44%
Episódio 24: 71.11%
Episódio 25: 77.78%
Episódio 26: 46.67%
Episódio 27: 64.44%
Episódio 28: 57.78%
Episódio 29: 64.44%
Episódio 30: 40.00%
Episódio 31: 75.56%
Episódio 32: 77.78%
Episódio 33: 57.78%
Episódio 34: 33.33%
Episódio 35: 64.44%
Episódio 36: 68.89%
Episódio 37: 57.78%
Episódio 38: 66.67%
Episódio 39: 77.78%
Episódio 40: 53.33%
Episódio 41: 71.11%
Episódio 42: 68.89%
Episódio 43: 60.00%
Episódio 44: 77.78%
Episódio 45: 53.33%
Episódio 46: 57.78%
Episódio 47: 64.44%
Episódio 48: 60.00%
Episódio 49: 55.56%
Episódio 50: 48.89%
Episódio 

## 📊 Por que Precision = Recall = F1 = Acurácia?

**Problema:** Quando os labels são **remapeados** para [0, 1, 2] em cada episódio:
- Episódio 1: [melanoma=0, nevo=1, ceratose=2]
- Episódio 2: [nevo=0, basalioma=1, melanoma=2]  
- Episódio 3: [ceratose=0, melanoma=1, dermatofibroma=2]

Ao agregar 200 episódios, cada slot [0, 1, 2] recebe uma **distribuição uniforme** de todas as classes reais. Com amostragem aleatória, a matriz de confusão fica balanceada e **todas as métricas convergem para o mesmo valor**!

**Solução:** Manter os **labels originais** (classes reais) para calcular métricas que fazem sentido semântico.

In [ ]:
# Converter para arrays
all_predictions_remapped = np.array(all_predictions_remapped)
all_labels_remapped = np.array(all_labels_remapped)
all_predictions_original = np.array(all_predictions_original)
all_labels_original = np.array(all_labels_original)


# Obter nomes das classes do dataset
class_names = dataset.classes

# Salvar resultados usando as classes ORIGINAIS + nomes das classes
exp_dir = save_fewshot_results(
    experiment_name="deit_prototypical_fewshot",
    model_name=CONFIG['model_name'],
    metric_name="Similaridade Cosine",
    normalization="Normalizacao L2",
    accuracies=accuracies,
    n_way=CONFIG['n_way'],
    n_shot=CONFIG['n_shot'],
    n_query=CONFIG['n_query'],
    n_episodes=CONFIG['n_episodes'],
    device=CONFIG['device'],
    all_predictions=all_predictions_original, 
    all_labels=all_labels_original,             
    class_names=class_names                     
)


COMPARAÇÃO DE MÉTRICAS:

1. Métricas com labels REMAPEADOS (cada episódio = [0,1,2]):
   (Labels perdem significado semântico - não recomendado!)
   Precision (macro): 65.00%
   Recall (macro):    65.00%
   F1-Score (macro):  64.99%
   Acurácia:          65.00%
   → Todas iguais! Não têm significado real.

2. Métricas com labels ORIGINAIS (classes reais do dataset):
   (Métricas por classe real - correto!)
   Precision (macro): 64.60%
   Recall (macro):    64.90%
   F1-Score (macro):  64.64%
   Acurácia:          65.00%
   → Valores diferentes! Fazem sentido semântico.

3. Acurácia média dos episódios (método padrão few-shot):
   Acurácia:  65.00% ± 12.21%
   → Métrica correta para few-shot episódico!

MÉTRICAS DETALHADAS POR CLASSE (Labels Originais):

Classes do dataset: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

              precision    recall  f1-score   support

       akiec     0.7003    0.7364    0.7179      1320
         bcc     0.5820    0.5663    0.5740      1335
